In [3]:
import os
os.chdir('../')

In [22]:
import numpy as np
import torch
import torch.nn.functional as F
from tqdm import tqdm

root_dir = '/data/archive/sd-v1-4'
models = ['dpm_solver++', 'uni_pc_bh2', 'dpm_solver_v3', 'rbf_order2', 'rbf_order3']
steps = [5, 6, 8, 10, 12, 15, 20]
scales= [9.5]

for scale in scales:
    ref_dir = os.path.join(root_dir, f'dpm_solver++_steps200_scale{scale}')
    for step in steps:
        for model in models:
            comp_dir = os.path.join(root_dir, f'{model}_steps{step}_scale{scale}')
            if not os.path.exists(comp_dir):
                continue
                
            mse_list = []
            for i in range(2000):
                ref_file = os.path.join(ref_dir, f"{i}.pt")
                comp_file = os.path.join(comp_dir, f"{i}.pt")
                if not os.path.exists(ref_file) or not os.path.exists(comp_file):
                    continue
                ref_data = torch.load(ref_file)
                comp_data = torch.load(comp_file)
                mses = torch.sqrt(torch.mean((ref_data['image'] - comp_data['image']) ** 2, dim=[1, 2, 3]))
                mse_list.append(mses)
            mses = torch.cat(mse_list, dim=0)
            print(comp_dir, f"{torch.mean(mses).item():.4f}", f'N={len(mses)}')
        print("")
            

/data/archive/sd-v1-4/dpm_solver++_steps5_scale9.5 0.6793 N=10000
/data/archive/sd-v1-4/uni_pc_bh2_steps5_scale9.5 0.6955 N=10000
/data/archive/sd-v1-4/rbf_order2_steps5_scale9.5 0.6666 N=10000
/data/archive/sd-v1-4/rbf_order3_steps5_scale9.5 0.6661 N=10000

/data/archive/sd-v1-4/dpm_solver++_steps6_scale9.5 0.6471 N=10000
/data/archive/sd-v1-4/uni_pc_bh2_steps6_scale9.5 0.6659 N=10000
/data/archive/sd-v1-4/rbf_order2_steps6_scale9.5 0.6347 N=10000
/data/archive/sd-v1-4/rbf_order3_steps6_scale9.5 0.6327 N=10000

/data/archive/sd-v1-4/dpm_solver++_steps8_scale9.5 0.5950 N=10000
/data/archive/sd-v1-4/uni_pc_bh2_steps8_scale9.5 0.6160 N=10000
/data/archive/sd-v1-4/rbf_order2_steps8_scale9.5 0.5859 N=10000
/data/archive/sd-v1-4/rbf_order3_steps8_scale9.5 0.5842 N=10000

/data/archive/sd-v1-4/dpm_solver++_steps10_scale9.5 0.5525 N=10000
/data/archive/sd-v1-4/uni_pc_bh2_steps10_scale9.5 0.5715 N=10000
/data/archive/sd-v1-4/rbf_order2_steps10_scale9.5 0.5487 N=10000
/data/archive/sd-v1-4/rbf_